In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"

FEATURE_GROUP_NAME = "time_series_hourly_feature_group"  
FEATURE_GROUP_VERSION = 1  

def fetch_hourly_rides(hours):
    # Calculate the current hour offset
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor("h")

    # Connect to Hopsworks manually
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
    fs = project.get_feature_store()

    # Get the feature group manually (no config dependency)
    fg = fs.get_feature_group(name=FEATURE_GROUP_NAME, version=FEATURE_GROUP_VERSION)

    # Query feature group with time filter
    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

# Example usage
rides_data = fetch_hourly_rides(3)  # Fetch rides from the last 3 hours
print(rides_data)


2025-03-05 04:19:04,125 INFO: Initializing external client
2025-03-05 04:19:04,125 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:19:04,855 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.37s) 
                   pickup_hour  pickup_location_id  rides
0    2025-03-05 01:00:00+00:00                 261      1
1    2025-03-05 01:00:00+00:00                 252      0
2    2025-03-05 02:00:00+00:00                  75      1
3    2025-03-05 01:00:00+00:00                 249     36
4    2025-03-05 02:00:00+00:00                 251      0
...                        ...                 ...    ...
2011 2025-03-05 08:00:00+00:00                  49      4
2012 2025-03-05 08:00:00+00:00                  43     87
2013 2025-03-05 08:00:00+00:00                 230    130
2014 2025-03-05 08:00:00+00:00                  75     48
2015 2025

In [4]:
# import hopsworks
# import pandas as pd
# from datetime import timedelta
# from src.inference import get_feature_store, fetch_predictions

# def fetch_hourly_rides(hours):
#     current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

#     fs = get_feature_store()
#     fg = fs.get_feature_group(
#         name=config.FEATURE_GROUP_NAME,
#         version=1
#     )

#     query = fg.select_all()
#     query = query.filter(fg.pickup_hour >= current_hour)

#     return query.read()

In [5]:
df = fetch_hourly_rides(12)

2025-03-05 04:19:11,986 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:19:11,989 INFO: Initializing external client
2025-03-05 04:19:11,989 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:19:12,567 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.58s) 


In [6]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 16:00:00+00:00,83,0
1,2025-03-04 20:00:00+00:00,74,7
2,2025-03-04 17:00:00+00:00,221,0
3,2025-03-04 20:00:00+00:00,253,0
4,2025-03-04 18:00:00+00:00,56,1
...,...,...,...
4279,2025-03-05 08:00:00+00:00,49,4
4280,2025-03-05 08:00:00+00:00,43,87
4281,2025-03-05 08:00:00+00:00,230,130
4282,2025-03-05 08:00:00+00:00,75,48


In [7]:
df_pred = fetch_predictions(12)

2025-03-05 04:19:22,939 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:19:22,942 INFO: Initializing external client
2025-03-05 04:19:22,942 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:19:23,524 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 


In [8]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,229,86.0,2025-03-05 00:00:00+00:00
1,85,0.0,2025-03-05 00:00:00+00:00
2,36,0.0,2025-03-05 00:00:00+00:00
3,252,0.0,2025-03-05 00:00:00+00:00
4,133,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
2507,144,39.0,2025-03-05 10:00:00+00:00
2508,178,0.0,2025-03-05 10:00:00+00:00
2509,29,0.0,2025-03-05 10:00:00+00:00
2510,43,84.0,2025-03-05 10:00:00+00:00


In [9]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [10]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-05 00:00:00+00:00,38,0,0.0
1,2025-03-05 00:00:00+00:00,47,0,0.0
2,2025-03-05 00:00:00+00:00,50,7,30.0
3,2025-03-05 00:00:00+00:00,260,1,1.0
4,2025-03-05 00:00:00+00:00,32,0,0.0
...,...,...,...,...
2003,2025-03-05 08:00:00+00:00,49,4,1.0
2004,2025-03-05 08:00:00+00:00,43,87,80.0
2005,2025-03-05 08:00:00+00:00,230,130,54.0
2006,2025-03-05 08:00:00+00:00,75,48,32.0


In [11]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [12]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
202,2025-03-05 00:00:00+00:00,2,0,0.0,0.0
470,2025-03-05 01:00:00+00:00,2,0,0.0,0.0
652,2025-03-05 03:00:00+00:00,2,0,0.0,0.0
913,2025-03-05 04:00:00+00:00,2,0,0.0,0.0
1112,2025-03-05 05:00:00+00:00,2,0,0.0,0.0
...,...,...,...,...,...
888,2025-03-05 04:00:00+00:00,263,4,75.0,71.0
1240,2025-03-05 05:00:00+00:00,263,19,40.0,21.0
1383,2025-03-05 06:00:00+00:00,263,70,37.0,-33.0
1587,2025-03-05 07:00:00+00:00,263,128,82.0,-46.0


In [13]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-05 00:00:00+00:00,38,0,0.0,0.0
1,2025-03-05 00:00:00+00:00,47,0,0.0,0.0
2,2025-03-05 00:00:00+00:00,50,7,30.0,23.0
3,2025-03-05 00:00:00+00:00,260,1,1.0,0.0
4,2025-03-05 00:00:00+00:00,32,0,0.0,0.0
...,...,...,...,...,...
2003,2025-03-05 08:00:00+00:00,49,4,1.0,-3.0
2004,2025-03-05 08:00:00+00:00,43,87,80.0,-7.0
2005,2025-03-05 08:00:00+00:00,230,130,54.0,-76.0
2006,2025-03-05 08:00:00+00:00,75,48,32.0,-16.0


In [14]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [15]:
mae_by_hour["MAE"].mean()

11.221115537848606